# CSV 파일 데이터 로드하기

TensorFlow에서 CSV 데이터를 사용하는 방법을 살펴보자.  

가장 간단한 방법은 csv 파일을 pandas dataframe으로 읽어들여 numpy배열로 처리하는 방법이다. 

In [1]:
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers

다음 예제는 전복(abalone)과 관련된 데이터세트이다. 
pd.read_csv() 함수를 사용하여 데이터를 읽어 들인다. 

In [2]:
abalone_file = tf.keras.utils.get_file("abalone_train.csv", "https://storage.googleapis.com/download.tensorflow.org/data/abalone_train.csv")

In [3]:
abalone = pd.read_csv(
    abalone_file,
    names=["Length", "Diameter", "Height", "Whole weight", "Shucked weight",
           "Viscera weight", "Shell weight", "Age"])

abalone.head()

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Age
0,0.435,0.335,0.110,0.334,0.1355,0.0775,0.0965,7
1,0.585,0.450,0.125,0.874,0.3545,0.2075,0.2250,6
2,0.655,0.510,0.160,1.092,0.3960,0.2825,0.3700,14
3,0.545,0.425,0.125,0.768,0.2940,0.1495,0.2600,16
4,0.545,0.420,0.130,0.879,0.3740,0.1695,0.2300,13


데이터 세트의 feature를 사용하여 전복의 나이(Age) 를 예측하는 것이므로 다음과 같이 훈련을 위해 feature와 label을 분리한다.

In [4]:
abalone_features = abalone.values[:, :-1]
abalone_labels = abalone.Age.values

회귀 모델로 나이를 예측한다. 여기에서는 keras.Sequential 모델이면 충분하다.

In [5]:
abalone_model = tf.keras.Sequential([
  layers.Dense(64),
  layers.Dense(1)
])

abalone_model.compile(loss = tf.keras.losses.MeanSquaredError(),
                      optimizer = tf.keras.optimizers.Adam())

In [6]:
abalone_model.fit(abalone_features, abalone_labels, epochs=5)

Epoch 1/5
104/104 [==============================] - 1s 1ms/step - loss: 64.7601
Epoch 2/5
104/104 [==============================] - 0s 1ms/step - loss: 12.5440
Epoch 3/5
104/104 [==============================] - 0s 1ms/step - loss: 8.8614
Epoch 4/5
104/104 [==============================] - 0s 1ms/step - loss: 8.2908
Epoch 5/5
104/104 [==============================] - 0s 1ms/step - loss: 7.7841


# tf.data 사용하기

tf.data API를 사용하면 간단하게 입력 데이터를 처리할 수 있다.  tf.data API를 사용하면 많은 양의 데이터를 처리하고 여러 데이터 형식에서 데이터를 읽으며 복잡한 변환을 수행할 수 있다.

- [tf.data.Dataset.from_tensor_slices()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices) : 주어진 데이터 소스의 slice로 데이터세트를 생성한다. 첫 dimension 으로 나뉘어 진다. 
- tf.data.Dataset은 기본적으로 파이썬의 반복 가능 객체이다. 즉 iterator로 꺼내쓸 수 있다.

In [7]:
# Slicing a 1D tensor produces scalar tensor elements.
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
for x in dataset:
    print(x)

tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)


In [8]:
# Slicing a 2D tensor produces 1D tensor elements.
dataset = tf.data.Dataset.from_tensor_slices([[1, 2], [3, 4]]) #2행 2열
for x in dataset:
    print(x)

tf.Tensor([1 2], shape=(2,), dtype=int32)
tf.Tensor([3 4], shape=(2,), dtype=int32)


In [9]:
# Two tensors can be combined into one Dataset object.
features = tf.constant([[1, 3], [2, 1], [3, 3]]) # ==> 3x2 tensor
labels = tf.constant(['A', 'B', 'A']) # ==> 3x1 tensor
dataset = tf.data.Dataset.from_tensor_slices((features, labels))
for x in dataset:
    print(x)

(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 3])>, <tf.Tensor: shape=(), dtype=string, numpy=b'A'>)
(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([2, 1])>, <tf.Tensor: shape=(), dtype=string, numpy=b'B'>)
(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([3, 3])>, <tf.Tensor: shape=(), dtype=string, numpy=b'A'>)


### 데이터프레임과 tf.data와 함께 사용하기
전복데이터를 데이터세트로 만든다.  

Dataset.from_tensor_slices 메서드는 DataFrame의 row을 반복하는 데이터세트를 생성한다.  
모델을 훈련시키려면 (inputs, labels) 쌍이 필요하므로 (features, labels)을 전달하면 Dataset.from_tensor_slices가 필요한 슬라이스 쌍을 반환한다.

In [10]:
abalone.head()

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Age
0,0.435,0.335,0.110,0.334,0.1355,0.0775,0.0965,7
1,0.585,0.450,0.125,0.874,0.3545,0.2075,0.2250,6
2,0.655,0.510,0.160,1.092,0.3960,0.2825,0.3700,14
3,0.545,0.425,0.125,0.768,0.2940,0.1495,0.2600,16
4,0.545,0.420,0.130,0.879,0.3740,0.1695,0.2300,13


In [11]:
features = abalone.iloc[:,:7]
features.head()

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight
0,0.435,0.335,0.110,0.334,0.1355,0.0775,0.0965
1,0.585,0.450,0.125,0.874,0.3545,0.2075,0.2250
2,0.655,0.510,0.160,1.092,0.3960,0.2825,0.3700
3,0.545,0.425,0.125,0.768,0.2940,0.1495,0.2600
4,0.545,0.420,0.130,0.879,0.3740,0.1695,0.2300


In [12]:
target = abalone.iloc[:,7]
target.head()

0     7
1     6
2    14
3    16
4    13
Name: Age, dtype: int64

In [13]:
dataset = tf.data.Dataset.from_tensor_slices((features, target))

# 데이터세트의 처음 3개를 출력한다.
for row in dataset.take(3):
    print(row)

(<tf.Tensor: shape=(7,), dtype=float64, numpy=array([0.435, 0.335, 0.11 , 0.334, 0.136, 0.077, 0.097])>, <tf.Tensor: shape=(), dtype=int64, numpy=7>)
(<tf.Tensor: shape=(7,), dtype=float64, numpy=array([0.585, 0.45 , 0.125, 0.874, 0.354, 0.207, 0.225])>, <tf.Tensor: shape=(), dtype=int64, numpy=6>)
(<tf.Tensor: shape=(7,), dtype=float64, numpy=array([0.655, 0.51 , 0.16 , 1.092, 0.396, 0.282, 0.37 ])>, <tf.Tensor: shape=(), dtype=int64, numpy=14>)


- **shuffle(buffer_size)** : dataset을 섞는다. buffer_size만큼 가져와서 섞기 때문에 잘 섞을려면 전체 dataset의 크기보다 큰 buffer_size가 필요하다. 즉 전체 데이터셋보다 큰 buffer_size가 이상적이긴하니 dataset와 컴퓨터 사양에 맞게 적절히 타협해서 설정하면 된다.
- **batch(batch_size)** : 주어진 크기로 데이터 세트를 자동으로 처리하는 batch(BATCH_SIZE)메서드를 사용할 수 있다. 기본 batch size는 1이다.

In [14]:
BUFFER_SIZE = len(dataset)
BATCH_SIZE = 10

In [15]:
datasets = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
for x, y in datasets.take(1):
    print(x)
    print()
    print(y)

tf.Tensor(
[[0.775 0.57  0.22  2.032 0.735 0.475 0.658]
 [0.515 0.43  0.14  0.834 0.367 0.2   0.23 ]
 [0.345 0.275 0.095 0.2   0.075 0.053 0.07 ]
 [0.37  0.29  0.09  0.244 0.089 0.066 0.075]
 [0.52  0.38  0.135 0.583 0.251 0.157 0.175]
 [0.645 0.5   0.16  1.246 0.547 0.327 0.3  ]
 [0.46  0.365 0.115 0.511 0.236 0.118 0.123]
 [0.415 0.315 0.115 0.39  0.202 0.065 0.103]
 [0.595 0.47  0.165 1.108 0.491 0.233 0.335]
 [0.57  0.45  0.135 0.78  0.335 0.185 0.21 ]], shape=(10, 7), dtype=float64)

tf.Tensor([17  8  6  7  8 10  7  9  9  8], shape=(10,), dtype=int64)


In [16]:
abalone_model.fit(datasets, epochs=5)

Epoch 1/5
332/332 [==============================] - 1s 1ms/step - loss: 7.1078
Epoch 2/5
332/332 [==============================] - 0s 1ms/step - loss: 6.5682
Epoch 3/5
332/332 [==============================] - 0s 1ms/step - loss: 6.3377
Epoch 4/5
332/332 [==============================] - 0s 1ms/step - loss: 6.1183
Epoch 5/5
332/332 [==============================] - 0s 1ms/step - loss: 5.9130


### tf.data.experimental.make_csv_dataset 함수

https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_csv_dataset 


tf.data 모듈 하나 이상의 CSV 파일에서 레코드를 추출하는 메서드를 제공한다.

tf.data.experimental.make_csv_dataset 함수는 csv 파일 세트를 읽기 위한 고급 인터페이스이다. CSV파일을 튜플 형식으로 생긴 batch 데이터셋으로 만들어 준다. 열 형식 유추와 일괄 처리 및 셔플링과 같은 많은 기능을 지원하며 사용이 간편하다.

In [17]:
abalone_batches = tf.data.experimental.make_csv_dataset(
    abalone_file, 
    header=False,
    column_names = ["Length", "Diameter", "Height", "Whole weight", "Shucked weight",
           "Viscera weight", "Shell weight", "Age"],
    batch_size=4,
    num_epochs=1,
    label_name="Age"
)

In [18]:
for feature_batch, label_batch in abalone_batches.take(1):
    for key, value in feature_batch.items():
        print("{:20s}: {}".format(key, value))
    print()
    print("label: ",label_batch)

Length              : [0.475 0.66  0.28  0.465]
Diameter            : [0.37  0.5   0.21  0.345]
Height              : [0.125 0.155 0.08  0.105]
Whole weight        : [0.655 1.377 0.108 0.401]
Shucked weight      : [0.266 0.649 0.041 0.242]
Viscera weight      : [0.172 0.288 0.026 0.034]
Shell weight        : [0.185 0.335 0.034 0.109]

label:  tf.Tensor([10 12  7  6], shape=(4,), dtype=int32)


tf.stack 함수를 통해 (features, label) 형태의 텐서로 packing한다.

In [19]:
def pack(feature, label):
    return tf.stack(list(feature.values()), axis=1), label

packed_dataset = abalone_batches.map(pack)

In [20]:
for feature, label in packed_dataset.take(1):
    print(feature)
    print()
    print(label)

tf.Tensor(
[[0.655 0.505 0.165 1.27  0.604 0.262 0.335]
 [0.74  0.575 0.22  2.012 0.891 0.526 0.471]
 [0.475 0.365 0.105 0.417 0.164 0.099 0.127]
 [0.47  0.355 0.115 0.415 0.167 0.084 0.139]], shape=(4, 7), dtype=float32)

tf.Tensor([10 12  7  7], shape=(4,), dtype=int32)


In [22]:
abalone_model.fit(packed_dataset, epochs=5)

Epoch 1/5
830/830 [==============================] - 2s 2ms/step - loss: 5.6466
Epoch 2/5
830/830 [==============================] - 1s 1ms/step - loss: 5.3157
Epoch 3/5
830/830 [==============================] - 1s 1ms/step - loss: 5.1516
Epoch 4/5
830/830 [==============================] - 1s 1ms/step - loss: 5.0402
Epoch 5/5
830/830 [==============================] - 1s 1ms/step - loss: 5.0053
